In [ ]:
# train_spell_error.txt
# train_clean.txt
spell_error = open("train_spell_error.txt", "r")
clean = open("train_clean.txt", "r")
spell_error = spell_error.read().split("\n")
clean = clean.read().split("\n")

# 将spell_error和clean的数据每个句子以空格切分，视为单词，每个单词最后一个元素为“:”则将其单独切分
for i in range(len(spell_error)):
    temp = spell_error[i].split(" ")
    spell_error[i] = []
    for j in range(len(temp)):
        # print(temp)
        # print('*',temp[j],'*')
        if len(temp[j]) == 0:
            continue
        if temp[j][-1] == ":":
            spell_error[i].append(temp[j][:-1])
            spell_error[i].append(":")
        else:
            spell_error[i].append(temp[j])

for i in range(len(clean)):
    temp = clean[i].split(" ")
    clean[i] = []
    for j in range(len(temp)):
        if len(temp[j]) == 0:
            continue
        if temp[j][-1] == ":":
            clean[i].append(temp[j][:-1])
            clean[i].append(":")
        else:
            clean[i].append(temp[j])

for i in range(len(spell_error)):
    if len(spell_error[i]) != len(clean[i]):
        for j in range(len(spell_error[i])):
            # 如果含有“:”且不是最后一个元素
            if spell_error[i][j].find(":") != -1 and spell_error[i][j][-1] != ":":
                # spell_error[i][j]位置赋予删除“:”后的单词
                spell_error[i][j] = spell_error[i][j].replace(":", "")
                # spell_error[i][j]之后插入“:”
                spell_error[i].insert(j+1, ":")

                

tol = 0  #79791
onesenhavemulerrors = 0   #22883
senlenserr = 0  #34
for i in range(len(spell_error)):
    indexx = 0
    for  j in range(len(spell_error[i])):
        if len(spell_error[i]) == len(clean[i]) and spell_error[i][j] != clean[i][j]:
            tol += 1
            indexx += 1
            # print(spell_error[i][j], clean[i][j])
            print(spell_error[i])
            print(clean[i])
            print(" ".join(clean[i]))
            print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
    if len(spell_error[i]) != len(clean[i]):
        # print(spell_error[i])
        # print(clean[i])
        # print('@@@@@@@@@@@@@@@@@@@@@@@@@@@@@')
        senlenserr +=1 
    if indexx > 1:
        onesenhavemulerrors += 1


# # 寻找所有":"之前的单词  
# te = {}
# for i in range(len(clean)):
#     for j in range(len(clean[i])-1):
#         if clean[i][j+1] == ":":
#             if clean[i][j] not in te:
#                 te[clean[i][j]] = 1
#             else:
#                 te[clean[i][j]] += 1

# for i in range(len(clean)):
#     for j in range(len(clean[i])):
#         if clean[i][j] in te and len(clean[i]) > j+1:
#             print(clean[i][j+1])
